In [2]:
import pandas as pd


I part 2 starter vi ud med at indlæse vores 'train_set.csv', 'validation_set.csv' og 'test_set.csv'

In [3]:
train_data = pd.read_csv('train_set.csv')
val_data = pd.read_csv('validation_set.csv')
test_data = pd.read_csv('test_set.csv')

Vi vil nu konvertere 'type' kolonnen til en binær klassificering, hvor 1 er lig med alle de reliable news og 0 er lig med alle de fake news. Samtidig med dette opretter vi en baseline model, som vi har valgt til at være naive bayes. 

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
import pandas as pd

# Indlæs træningsdata
train_data = pd.read_csv('train_set.csv')

# Erstat NaN-værdier i 'stemmed_content' med en tom streng
train_data['stemmed_content'].fillna('', inplace=True)

# Definer dine kategorier
reliable_categories = ['reliable', 'political']
fake_categories = ['fake', 'bias', 'conspiracy', 'hate', 'junksci', 'rumor', 'unreliable', 'satire']

# Opret en funktion, der omdanner 'type' til binære værdier
def map_to_binary(article_type):
    if article_type in reliable_categories:
        return 1  # 'reliable'
    elif article_type in fake_categories:
        return 0  # 'fake'
    else:
        return 0  # Sikrer, at funktionen returnerer 0 for alle andre tilfælde, hvilket eliminerer risikoen for NaN-værdier

# Opdater 'type' kolonnen i 'train_data' med de binære værdier
train_data['type'] = train_data['type'].apply(map_to_binary)

# Nu er 'train_data['type']' klar til brug som 'y_train'
y_train = train_data['type']
X_train = train_data['stemmed_content']

# Opret en pipeline med TF-IDF og Naive Bayes
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('classifier', MultinomialNB())
])

# Træn modellen på træningsdata
pipeline.fit(X_train, y_train)

# Her er den del hvor vi indlæser valideringsdata og forudsiger
# Antager at du har en 'val_set.csv' som valideringsdatasæt
val_data = pd.read_csv('validation_set.csv')
val_data['stemmed_content'].fillna('', inplace=True)

# Opdater 'type' kolonnen i 'val_data' med de binære værdier
val_data['type'] = val_data['type'].apply(map_to_binary)

X_val = val_data['stemmed_content']
y_val = val_data['type']

# Brug pipelinen til at forudsige på valideringsdata
y_pred = pipeline.predict(X_val)

# Beregn og udskriv præcision for valideringsdata
val_accuracy = accuracy_score(y_val, y_pred)
print(f"Validation accuracy: {val_accuracy}")


Validation accuracy: 0.8293869346733669


Beregner også F1_Score, Recall og Precision:

In [15]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Beregn præcision, recall og F1-score
precision = precision_score(y_val, y_pred)
recall = recall_score(y_val, y_pred)
f1 = f1_score(y_val, y_pred)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")


Precision: 0.8771
Recall: 0.6841
F1-Score: 0.7686


# Task 2

Iforhold til meta data overvejede vi domain men i og med vi tidligere fandt ud af at hver domain kun udgiver 1 type artikel mener vi at det vil "overfitte" vores model, hvor vores model bare genkender domain istedet for at forudsige. Vi valgte derfor i stedet at prøve med authors og for at om dette spillede nogen rolle. Det viser sig at accuracy stiger med ca. 10%, med author som meta-feature rammer vi nu en accuracy på 92,2 %. 

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score
import pandas as pd

# Antager at train_data og val_data allerede er indlæst

# Forberedelse af data (Erstat NaN-værdier og anvend map_to_binary funktionen)
train_data['stemmed_content'].fillna('', inplace=True)
train_data['authors'].fillna('', inplace=True)
val_data['stemmed_content'].fillna('', inplace=True)
val_data['authors'].fillna('', inplace=True)

y_train = train_data['type'].apply(map_to_binary)
y_val = val_data['type'].apply(map_to_binary)

# Forbered X_train og X_val som dictionaries til ColumnTransformer
X_train = train_data[['stemmed_content', 'authors']]
X_val = val_data[['stemmed_content', 'authors']]

# Opret en ColumnTransformer til at forarbejde de forskellige kolonner
preprocessor = ColumnTransformer(
    transformers=[
        ('text', TfidfVectorizer(), 'stemmed_content'),
        ('authors', OneHotEncoder(handle_unknown='ignore'), ['authors'])
    ]
)

# Opret en pipeline med preprocessor og Naive Bayes
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', MultinomialNB())
])

# Træn modellen på træningsdata
pipeline.fit(X_train, y_train)

# Forudsige på valideringsdata
y_pred = pipeline.predict(X_val)

# Beregn og udskriv præcision for valideringsdata
val_accuracy = accuracy_score(y_val, y_pred)
print(f"Validation accuracy: {val_accuracy}")

mse_logistic = mean_squared_error(y_val, y_pred_probs)
print(f"LogisticRegression MSE: {mse_logistic}")

Validation accuracy: 0.9220603015075377
LogisticRegression MSE: 0.08537117598564649


# task 3

Jeg starter med at indlæse filen fra exercise 2 - scraped_articles.csv

In [6]:
scraped_articles = pd.read_csv('scraped_articles.csv')

Da disse artikler ikke er blevet renset eller lavet stemming på endnu, vil vi starte med at gøre dette:

In [7]:
import pandas as pd
import re

def clean_text(text):
    if pd.isna(text):  # Tjekker for NaN værdier og konverterer dem til en tom streng
        return ''
    text = str(text).lower()  # Sikrer at tekst er en streng og konverterer til små bogstaver
    text = re.sub(r'\s+', ' ', text)  # Erstatter al hvidplads med et enkelt mellemrum
    text = re.sub(r'\b\d{4}-\d{2}-\d{2}\b', '<DATE>', text)  # Erstatter datoer med pladsholder
    text = re.sub(r'\d+', '<NUM>', text)  # Erstatter tal med pladsholder
    text = re.sub(r'\S+@\S+', '<EMAIL>', text)  # Erstatter e-mails med pladsholder
    text = re.sub(r'https?://\S+', '<URL>', text)  # Erstatter URL'er med pladsholder
    text = re.sub(r'[^\w\s]', '', text)  # Fjerner tegnsætning
    return text

# Path to the scraped articles CSV file
csv_file_path = 'scraped_articles.csv'  # Erstat med din faktiske filsti

# Load the scraped articles data from the CSV file
scraped_articles = pd.read_csv(csv_file_path, dtype=str)

# Apply the cleaning function to the 'text' column which contains the article text
scraped_articles['cleaned_text'] = scraped_articles['text'].apply(clean_text)

# Keeping only the 'cleaned_text' column for further processing
cleaned_scraped_articles = scraped_articles[['cleaned_text']]

# Optional: Save the cleaned data to a new CSV file if needed
# cleaned_scraped_articles.to_csv('path_to_your_file/cleaned_scraped_articles.csv', index=False)
cleaned_scraped_articles.head


<bound method NDFrame.head of                                            cleaned_text
0     we know whats coming east ukraine braces for r...
1     in eastern ukraine the tide of this war hasnt ...
2     in order to preserve life and encirclement i h...
3     fighting has been raging in ukraine for two ye...
4     deadly explosions have rocked ukraines souther...
...                                                 ...
5762  indias capital delhi has banned motorbike taxi...
5763  two years ago indian prime minister narendra m...
5764  the discovery of two charred bodies in a burnt...
5765  the judges of masterchef india have been criti...
5766  ruchelle barrie remembers feeling out of place...

[5767 rows x 1 columns]>

In [11]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

# Download stopwords from NLTK
nltk.download('punkt')
nltk.download('stopwords')

# Load stop words
stop_words = set(stopwords.words('english'))

# Initialize the stemmer
stemmer = PorterStemmer()

def remove_stopwords_and_stem(text):
    # Tokenize the text into words
    words = word_tokenize(text)
    # Remove stop words and stem each word
    return ' '.join([stemmer.stem(word) for word in words if word not in stop_words])

# Apply the function to remove stop words and stem the words in the 'cleaned_text' column
# Brug .loc til at undgå SettingWithCopyWarning
cleaned_scraped_articles.loc[:, 'stemmed_content'] = cleaned_scraped_articles['cleaned_text'].apply(remove_stopwords_and_stem)

# Vis de første par rækker for at bekræfte ændringerne
cleaned_scraped_articles.head()



[nltk_data] Downloading package punkt to /Users/bruger/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/bruger/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,cleaned_text,stemmed_content,type
0,we know whats coming east ukraine braces for r...,know what come east ukrain brace russian advan...,1
1,in eastern ukraine the tide of this war hasnt ...,eastern ukrain tide war hasnt chang come fast ...,1
2,in order to preserve life and encirclement i h...,order preserv life encircl withdrawn unit avdi...,1
3,fighting has been raging in ukraine for two ye...,fight rage ukrain two year sinc russia invas m...,1
4,deadly explosions have rocked ukraines souther...,deadli explos rock ukrain southern port citi o...,1


Nu har vi altså renset og stemmed vores scraped_articles.csv og lavet en ny kolonne i dataen, som hedder cleaned_stemmed_text. Det næste vi vil gøre er at oprette en kolonne 'type' som fortæller om artiklen er reliable eller fake. Her vil vi vurdere alle artiklerne til at være reliable, da de kommer fra bbc. Dette vil altså sige at der kommer til at stå et 1 til under 'type' for alle artiklerne, da det betyder reliable i vores binære klassificering. 

In [12]:
cleaned_scraped_articles['type'] = 1

cleaned_scraped_articles.to_csv('cleaned_scraped_articled')

cleaned_scraped_articles



/var/folders/78/tggs5x_100l2bhsl5ydp24mh0000gn/T/ipykernel_13165/2909232008.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_scraped_articles['type'] = 1


,cleaned_text,stemmed_content,type
0,we know whats coming east ukraine braces for r...,know what come east ukrain brace russian advan...,1
1,in eastern ukraine the tide of this war hasnt ...,eastern ukrain tide war hasnt chang come fast ...,1
2,in order to preserve life and encirclement i h...,order preserv life encircl withdrawn unit avdi...,1
3,fighting has been raging in ukraine for two ye...,fight rage ukrain two year sinc russia invas m...,1
4,deadly explosions have rocked ukraines souther...,deadli explos rock ukrain southern port citi o...,1
...,...,...,...
5762,indias capital delhi has banned motorbike taxi...,india capit delhi ban motorbik taxi road deal ...,1
5763,two years ago indian prime minister narendra m...,two year ago indian prime minist narendra modi...,1
5764,the discovery of two charred bodies in a burnt...,discoveri two char bodi burnt vehicl india har...,1
5765,the judges of masterchef india have been criti...,judg masterchef india criticis social media le...,1


Nu hvor vi har en kolonne med stemmed_content hvor alle er sat til reliable under type, vil vi sammensætte vores train_data og cleaned_scraped_articles. 

In [17]:
train_scraped_combined = pd.concat([train_data, cleaned_scraped_articles], ignore_index=True)

# Vis de første par rækker af den nye kombinerede DataFrame for at bekræfte, at alt ser korrekt ud
train_scraped_combined.head()

,Unnamed: 0,id,domain,type,url,content,scraped_at,inserted_at,updated_at,title,authors,keywords,meta_keywords,meta_description,tags,summary,source,tokens,stemmed_content,cleaned_text
0,NUM,NUMNUM,abovetopsecretcom,0,URL,geoengineering cost analysis you lolyes seen ...,NUMNUMNUMtNUMNUMNUMNUM,DATE NUMNUMNUMNUM,DATE NUMNUMNUMNUM,possible weather mod setup page NUM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,geoengineering cost analysis lolyes seen docum...,geoengin cost analysi loly seen documentwhat t...,NaN
1,NUM,NUMNUM,thelibertybeaconcom,0,URL,lawNUM philadelphia february NUM NUM NUMNUM pm...,NUMNUMNUMtNUMNUMNUMNUM,DATE NUMNUMNUMNUM,DATE NUMNUMNUMNUM,merck sued in philly over shingles vaccine inj...,NaN,NaN,NaN,NaN,natural immunity chickenpox merck lawsuit vacc...,NaN,NaN,lawNUM philadelphia february NUM NUM NUMNUM pm...,lawnum philadelphia februari num num numnum pm...,NaN
2,NUM,NUM,newswithviewscom,0,URL,additional titles our right to attack iraq by ...,DATE NUMNUMNUMNUM,DATE NUMNUMNUMNUM,DATE NUMNUMNUMNUM,devvy kidd new gun bill in congress more unco...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,additional titles right attack iraq devvy kidd...,addit titl right attack iraq devvi kidd februa...,NaN
3,NUM,NUM,nytimescom,1,URL,i dont know who takes him said an eastern conf...,DATE NUMNUMNUMNUM,DATE NUMNUMNUMNUM,DATE NUMNUMNUMNUM,multiple nicknames dwindling openings,howard beck,NaN,oneal shaquille basketball free agents sports ...,shaquille oneal who once rumbled across the co...,NaN,NaN,nytimes,dont know takes said eastern conference scout ...,dont know take said eastern confer scout cite ...,NaN
4,NUM,NUM,nationalreviewcom,1,URL,plus one article on google plus thanks to ali ...,NUMNUMNUMtNUMNUMNUMNUM,DATE NUMNUMNUMNUM,DATE NUMNUMNUMNUM,iran news round up,NaN,NaN,national review national review online article,NaN,NaN,NaN,NaN,plus one article google plus thanks ali alfone...,plu one articl googl plu thank ali alfoneh ass...,NaN


In [33]:
train_scraped_combined.to_csv('train_scraped_combined.csv', index=False)


Nu prøver vi at bruge vores simple baseline "naive bayes" og ser om de 5767 ekstra artikler kan øge præcisionen.

In [18]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('classifier', MultinomialNB())
])

# Træn modellen på den nye kombinerede træningsdata
pipeline.fit(train_scraped_combined['stemmed_content'], train_scraped_combined['type'])

# Valideringsdatasættet forbliver det samme som før
# Husk at sikre, at 'stemmed_content' og 'type' kolonnerne er forberedt som før

# Brug pipelinen til at forudsige på valideringsdata
y_pred = pipeline.predict(X_val)

# Beregn og udskriv præcision for valideringsdata
val_accuracy = accuracy_score(y_val, y_pred)
print(f"Validation accuracy with additional reliable data: {val_accuracy}")

Validation accuracy with additional reliable data: 0.8320904522613065


Som vi kan se på resultatet er nøjagtigheden blevet øget med ca. 0,3% hvilket giver god mening efter som vi har tilføjet 5767 artikler til train_data som i forvejen bestod af 796000 artikler. Da det kun er 5767 artikler vi har tilføjet til train_data havde vi også forventet en meget lille forbedring i nøjagtigheden. Vi har altså kun øget træningssættet med 0,72% og dermed skulle det ikke forbedre nøjagtigheden vildt meget. 

Part 3

Bruger vores advanced model her, som resulterede i en logistic regression. 

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import precision_score, recall_score, f1_score


# Indlæs træningsdata

# Erstat NaN-værdier i 'stemmed_content' med en tom streng
train_data['stemmed_content'].fillna('', inplace=True)

# Definer dine kategorier
reliable_categories = ['reliable', 'political']
fake_categories = ['fake', 'conspiracy', 'hate', 'junksci', 'rumor', 'unreliable', 'satire','bias']

# Opret en funktion, der omdanner 'type' til binære værdier
def map_to_binary(article_type):
    if article_type in reliable_categories:
        return 1  # 'reliable'
    elif article_type in fake_categories:
        return 0  # 'fake'
    # Sikrer, at funktionen returnerer 0 for alle andre tilfælde, hvilket eliminerer risikoen for NaN-værdier
    return 0  

# Anvend denne funktion på dit datasæt for at oprette en binær label
y_train = train_data['type'].apply(map_to_binary)

# Nu er 'y_train' klar uden risiko for NaN-værdier
X_train = train_data['stemmed_content']  # 'X_train' er også klar til brug

# Opret en pipeline med TF-IDF og logistisk regression, øg antallet af iterationer
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('classifier', LogisticRegression(max_iter=10000))  
])

# Træn modellen på træningsdata
pipeline.fit(X_train, y_train)

# Her er den del hvor vi indlæser valideringsdata og forudsiger
# Antager at du har en 'val_set.csv' som valideringsdatasæt
val_data['stemmed_content'].fillna('', inplace=True)
X_val = val_data['stemmed_content']
y_val = val_data['type'].apply(map_to_binary)

# Brug pipeline til at forudsige på valideringsdata
y_pred = pipeline.predict(X_val)

# Forudsagte sandsynligheder (hvis du har brug for dem til MSE beregning)
y_pred_probs = pipeline.predict_proba(X_val)[:, 1]

# Beregn og udskriv præcision for valideringsdata
val_accuracy = accuracy_score(y_val, y_pred)
print(f"Validation accuracy: {val_accuracy}")

# Beregn MSE ved at sammenligne forudsagte sandsynligheder med de faktiske labels
# Bemærk: Dette er ikke standard for klassificering og anvendes normalt i regression.
mse_logistic = mean_squared_error(y_val, y_pred_probs)
print(f"LogisticRegression MSE: {mse_logistic}")

Validation accuracy: 0.8791256281407035
LogisticRegression MSE: 0.08537117598564649


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Beregn præcision, recall og F1-score
precision = precision_score(y_val, y_pred)
recall = recall_score(y_val, y_pred)
f1 = f1_score(y_val, y_pred)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

Her tilføjer vi authors som en meta feature og kan se at modellen virker bedre nu og giver en bedre accuracy. 

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score, mean_squared_error, precision_score, recall_score, f1_score
import pandas as pd

# Antager at train_data og val_data allerede er indlæst

# Erstat NaN-værdier i 'stemmed_content' og 'domain' med en tom streng
train_data['stemmed_content'].fillna('', inplace=True)
train_data['authors'].fillna('', inplace=True)

val_data['stemmed_content'].fillna('', inplace=True)
val_data['authors'].fillna('', inplace=True)

# Definer dine kategorier
reliable_categories = ['reliable', 'political']
fake_categories = ['fake', 'bias', 'conspiracy', 'hate', 'junksci', 'rumor', 'unreliable', 'satire']

# Opret en funktion, der omdanner 'type' til binære værdier
def map_to_binary(article_type):
    return 1 if article_type in reliable_categories else 0

# Anvend denne funktion på dit datasæt for at oprette en binær label
y_train = train_data['type'].apply(map_to_binary)
y_val = val_data['type'].apply(map_to_binary)

# Forbered features
X_train = train_data[['stemmed_content', 'authors']]
X_val = val_data[['stemmed_content', 'authors']]

# Opret en ColumnTransformer til at forarbejde de forskellige kolonner
preprocessor = ColumnTransformer(
    transformers=[
        ('text', TfidfVectorizer(), 'stemmed_content'),
        ('authors', OneHotEncoder(handle_unknown='ignore'), ['authors'])
    ]
)

# Opret en pipeline med preprocessor og logistisk regression
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(max_iter=1000))
])

# Træn modellen på træningsdata
pipeline.fit(X_train, y_train)

# Forudsige på valideringsdata
y_pred = pipeline.predict(X_val)

# Beregn og udskriv præcision for valideringsdata
val_accuracy = accuracy_score(y_val, y_pred)
print(f"Validation accuracy: {val_accuracy}")

# Beregn MSE ved at sammenligne forudsagte sandsynligheder med de faktiske labels
y_pred_probs = pipeline.predict_proba(X_val)[:, 1]
mse_logistic = mean_squared_error(y_val, y_pred_probs)
print(f"LogisticRegression MSE: {mse_logistic}")

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Beregn præcision, recall og F1-score
precision = precision_score(y_val, y_pred)
recall = recall_score(y_val, y_pred)
f1 = f1_score(y_val, y_pred)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")